In [1]:
from data.process import *

2024-07-23 14:38:57.146489: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-23 14:38:57.155485: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 14:38:57.167220: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 14:38:57.167259: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 14:38:57.174628: I tensorflow/core/platform/cpu_feature_gua

In [2]:
train_df, dev_df, test_df = retrieve_train_dev_test_as_category_for_xgboost()
X_train, Y_train = split_input_output(train_df)
X_dev, Y_dev = split_input_output(dev_df)

In [3]:
from lightgbm import LGBMClassifier, log_evaluation
from model.lgbm_opt import *

/home/gakim/gabyproj/InsuranceCrossSelling/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
lgbm_params = {
    **lgbm_fixed_params,
    # categorical features
    'enable_categorical': True,
    # fixed after secone optimization
    'colsample_bytree': 0.21039136022674532,
    'learning_rate': 0.01,
    'max_depth': 14,
    'num_leaves': 221,
    # override fixed params
    'n_estimators': 5000,
    'early_stopping_rounds': 200,
}

In [5]:
cat_features = list(X_train.select_dtypes(include=['object']).columns)

In [6]:
model = LGBMClassifier(
    **lgbm_params,
)
log_callback = log_evaluation(period=50)
boosting = model.fit(X_train, Y_train, eval_set=[(X_dev, Y_dev)], categorical_feature=cat_features, eval_metric='auc', callbacks=[log_callback])

[50]	valid_0's auc: 0.862409	valid_0's binary_logloss: 0.336559
[100]	valid_0's auc: 0.868027	valid_0's binary_logloss: 0.310795
[150]	valid_0's auc: 0.869325	valid_0's binary_logloss: 0.295936
[200]	valid_0's auc: 0.870496	valid_0's binary_logloss: 0.284841
[250]	valid_0's auc: 0.87161	valid_0's binary_logloss: 0.277671
[300]	valid_0's auc: 0.872534	valid_0's binary_logloss: 0.272579
[350]	valid_0's auc: 0.873647	valid_0's binary_logloss: 0.268312
[400]	valid_0's auc: 0.87425	valid_0's binary_logloss: 0.265549
[450]	valid_0's auc: 0.875049	valid_0's binary_logloss: 0.262947
[500]	valid_0's auc: 0.875711	valid_0's binary_logloss: 0.260816
[550]	valid_0's auc: 0.876297	valid_0's binary_logloss: 0.259149
[600]	valid_0's auc: 0.876755	valid_0's binary_logloss: 0.257899
[650]	valid_0's auc: 0.877077	valid_0's binary_logloss: 0.257005
[700]	valid_0's auc: 0.877365	valid_0's binary_logloss: 0.256107
[750]	valid_0's auc: 0.877745	valid_0's binary_logloss: 0.255287
[800]	valid_0's auc: 0.87812

In [7]:
import pprint
pprint.pprint(model.best_score_)

defaultdict(<class 'collections.OrderedDict'>,
            {'valid_0': OrderedDict([('auc', 0.8844125872644719),
                                     ('binary_logloss', 0.2472496997001291)])})


In [8]:
# model.get_booster().save_model("xgboost_model_new.json")